## import modules

In [9]:
import pandas as pd
from scipy import stats
import numpy as np

## census data processing

In [10]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0:3,:]
census=census.loc[census.Level=='STATE']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

## urban-rural

In [11]:
c18=pd.read_excel('datasets/C-18.xlsx',engine='openpyxl',skiprows=6,header=None)

In [12]:
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)

In [13]:
# useful_data=[]
tri_list=[]
bi_list=[]
uni_list=[]
for i,state in enumerate(STATE_NAMES):
    
    # here i is the state code
    urban_pop=census[(census['State']==i) & (census['TRU']=='Urban')]['TOT_P'].values[0]
    rural_pop=census[(census['State']==i) & (census['TRU']=='Rural')]['TOT_P'].values[0]
    
    # tri
    tri_urban=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Urban')].iloc[0,8]
    tri_rural=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Rural')].iloc[0,8]

    #bi
    bi_urban=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Urban')].iloc[0,5] - tri_urban
    bi_rural=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]=='Total') & (c18.iloc[:,3]=='Rural')].iloc[0,5] - tri_rural

    #uni
    uni_urban=urban_pop-bi_urban-tri_urban
    uni_rural=rural_pop-bi_rural-tri_rural

   
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html   
    # p_value=stats.ttest_ind([urban_pop/rural_pop]*3,[tri_urban/tri_rural,bi_urban/bi_rural,uni_urban/uni_rural]).pvalue
    p_value=stats.ttest_1samp([tri_urban/tri_rural,bi_urban/bi_rural,uni_urban/uni_rural],[urban_pop/rural_pop]).pvalue[0]

    tri_item={
        'state-code':i,
        'urban-percentage':round(100*tri_urban/urban_pop,2),
        'rural-percentage':round(100*tri_rural/rural_pop,2),
        'p-value':p_value
    }

    bi_item={
        'state-code':i,
        'urban-percentage':round(100*bi_urban/urban_pop,2),
        'rural-percentage':round(100*bi_rural/rural_pop,2),
        'p-value':p_value
    }

    uni_item={
        'state-code':i,
        'urban-percentage':round(100*uni_urban/urban_pop,2),
        'rural-percentage':round(100*uni_rural/rural_pop,2),
        'p-value':p_value
    }
    tri_list.append(tri_item)
    bi_list.append(bi_item)
    uni_list.append(uni_item)

    # item={
    #     'state-code':i,
    #     'state-name':state,
    #     'p-value':p_value,
    #     'urban-to-rural':urban_pop/rural_pop,
    #     'tri-urban-to-rural-ratio':tri_urban/tri_rural,
    #     'bi-urban-to-rural-ratio':bi_urban/bi_rural,
    #     'uni-urban-to-rural-ratio':uni_urban/uni_rural
    # }

    # useful_data.append(item)

In [14]:
tri_df=pd.DataFrame(tri_list)
bi_df=pd.DataFrame(bi_list)
uni_df=pd.DataFrame(uni_list)

In [15]:
tri_df.to_csv('outputs/geography-india-c.csv',index=False)
bi_df.to_csv('outputs/geography-india-b.csv',index=False)
uni_df.to_csv('outputs/geography-india-a.csv',index=False)

#### results
- for no state or ut the ratio is significantly different